<a href="https://colab.research.google.com/github/sera0911/asia_ai_study/blob/main/Python_web_programming_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###day4 정리

#### 템플릿파일에서 다른 템플릿을 포함하기 위해 선언하는 템플릿 태그 :  
{%include "템플릿이름.html" %}  

#### 페이징 구현에 사용한 클래스 : Paginator  
페이징을 처리할 때 사용하는 Paginator 객체 속성  
.count - 전체 게시물 개수  
.per_page - 페이지당 보여줄 게시물 개수  
.page_range - 페이지 범위  
.has_previous, has_next - 전 후에 있는지
.number - 현재 페이지 번호  
.start_index, end_index -첫번호, 끝번호  

#### 템플릿에서 필터 기능 중 add는 기본 제공 필터,  
새로운 필터를 정의해서 템플릿에서 사용하려면 
1. 필터 함수 정의 - 앱폴더/templatetags 디렉토리/***_filter.py에 정의
2. 필터 등록할 템플릿의 라이브러리 객체에 필터 함수 등록 - @register.filter  
3. 정의한 필터 함수를 템플릿에 적용하기 위해 선언하는 템플릿 태그 - {% load ***_filter %}


#### 로그인 로그아웃 구현에 사용되는 장고 제공 기본 앱 - django.contrib.auth    
(User 모델[username, password], UserCreateForm[username, password1, password2], LoginView )  


장고 제공 기본 앱 django.contrib.auth 은 로그인 템플릿의 기본 위치 설정  
(registration/login.html)을 변경 => 로그인구현앱/urls.py에서  
path('login/', auth_views.LoginView.as_view(template_name='common/login.html'), name='login'),  

장고 제공 기본 앱 django.contrib.auth은 로그인 후에 redirect되는 템플릿의 기본 위치 설정  
(/accounts/profile/) 을 변경 => config/settings.py 파일에서 LOGIN_REDIRECT_URL 필드 추가

#### 폼에 발생하는 오류 종류 
- 필드 오류(입력값이 누락되었거나 형식에 맞지 않음)  
- 넌필드 오류(입력값과 관계없이 발생한 오류)  


#### 회원가입 기능 구현을 위해서 기본 앱 django.contrib.auth 사용  
회원가입 템플릿에서 입력한 값을 반환하는 함수 - from.cleaned_data.get("파라미터명")  
django.contrib.auth 앱에서 제공하는 인증과 로그인 함수 - authenticate, login  






### day5 시작  

#### 로그 아웃 구현
1. 네비게이션 바 수정하기

```
...
<li class="nav-item">
                {% if user.is_authenticated %}
                <a class="nav-link" href="{% url 'common:logout' %}">{{ user.username }} [로그아웃]</a>
                {% else %}
                <a class="nav-link" href="{% url 'common:login' %}">로그인</a>
            </li>
...
```

2. url요청과 뷰함수 매핑common/urls.py 에 path함수 추가


```
path('logout/', auth_views.LogoutView.as_view(), name='logout'),
```

3. 로그아웃 성공 시 리다이렉트할 위치 설정 config/settings.py 파일에 맨 아래에 LOGOUT_REDIRECT_URL = '/' 추가

```
LOGOUT_REDIRECT_URL = '/'
```



#### 질문과 답변 쓰기 저장에서 자동으로 글 작성자 입력되도록 처리  
1. Question, Answer 모델 - 글 작성자에 해당하는 author필드를 추가

```
from django.contrib.auth.models import User
...
#각 클래스 별 맨 아래에 author 추가
author = models.ForeignKey(User, on_delete=models.CASCADE)
```
django.contrib.auth 앱이 제공하는 모델(username, password)을  ForeignKey로 적용  

2. makemigrations 명령 실행하고 author 필드 추가 문제 해결

모델을 수정했으므로 makemigrations 명령과 migrate 명령을 실행  

cmd 입력
```
(mysite) c:\projects\mysite>python manage.py makemigrations

>>>2 #입력
```


migrate 명령으로 변경된 내용을 데이터베이스에 적용
```
(mysite) c:\projects\mysite>python manage.py migrate
```




#### author 필드 적용하기
1. 답변 등록 함수 수정

first/views.py에서 answer_create, question_create 수정
```
answer_create
...
answer.author = request.user
answer.save()

...
question_create
...
question.author = request.user
question.save()
...
```

로그아웃 상태에서 질문 또는 답변을 등록하면 다음과 같은 ValueError 오류가 발생  
request.user가 User 객체가 아닌 AnonymousUser 객체라서 발생  
request.user에는 로그아웃 상태이면 AnonymousUser 객체가, 로그인 상태이면 User 객체가 저장됨     
로그인이 필요한 함수 answer_create, question_create에 @login_required 애너테이션을 사용하여 로그인이 되었는지 우선 검사하여 오류를 방지한다  
 (=> 자동으로 로그인 화면으로 이동) 

views.py에서 수정
```
from django.contrib.auth.decorators import login_required

...
@login_required(login_url='common:login') #추가
def answer_create(request, question_id):
...
@login_required(login_url='common:login') #추가
def question_create(request):
...
```


#### 게시물 수정 & 삭제 기능 구현
1. 질문, 답변을 언제 수정했는지 확인할 수 있도록 Question 모델과 Answer 모델에 수정일시를 의미하는 modify_date 필드를 추가

first/models.py answer. question에 각각 추가
```
modify_date = models.DateTimeField(null=True, blank=True)
```
2. makemigrations, migrate 명령 수행
cmd에 각각 추가하기


```
(myweb) c:\workspace\project\mysite>python manage.py makemigrations
(myweb) c:\workspace\project\mysite>python manage.py migrate
```

#### 질문 수정 기능 추가하기
1. 질문 수정 기능 추가 question_detail.html 수정하기

```
<div>{{ question.create_date }}</div>
            </div>
        </div>
<!-- ------------------------------- [edit] -------------------------------- -->
        {% if request.user == question.author %}
        <div class="my-3">
            <a href="{% url 'pybo:question_modify' question.id  %}" 
               class="btn btn-sm btn-outline-secondary">수정</a>
        </div>
        {% endif %}
<!-- ----------------------------------------------------------------------- -->
    </div>
```

2. 질문 수정 버튼의 URL 매핑 추가

first/urls.py아래 추가
```
path('question/modify/<int:question_id>/', views.question_modify, name='question_modify'),
```

3. 질문 수정 뷰 함수 추가 

- 오류를 발생시키기 위해 messages 모듈을 이용(error(오류는 폼 필드와 관련이 없으므로 넌필드 오류))  

views.question_modify 함수를 추가  
first/views.py에 추가

```
from django.contrib import messages
...

@login_required(login_url='common:login')
def question_modify(request, question_id):
    """
    pybo 질문수정
    """
    question = get_object_or_404(Question, pk=question_id)
    if request.user != question.author:
        messages.error(request, '수정권한이 없습니다')
        return redirect('pybo:detail', question_id=question.id)

    if request.method == "POST":
        form = QuestionForm(request.POST, instance=question)
        if form.is_valid():
            question = form.save(commit=False)
            question.author = request.user
            question.modify_date = timezone.now()  # 수정일시 저장
            question.save()
            return redirect('pybo:detail', question_id=question.id)
    else:
        form = QuestionForm(instance=question)
    context = {'form': form}
    return render(request, 'pybo/question_form.html', context)
```


수정하기(get)요청 /first/question/modify/256 - 질문 수정 화면이 서비스되면서 수정해야 글의 데이터가 포함  
form = QuestionForm(instance=question)    
수정하기(post)요청 /first/question/modify/256 -  수정된 글 데이터를 전송받아서 DB에 저장    
form = QuestionForm(request.POST, instance=question)

4. 질문 상세 템플릿에서 오류 표시 기능 추가

question_detail.html 추가
```
{% extends 'base.html' %}
{% block content %}
<div class="container my-3">
<!-- 사용자오류 표시 -->
    {% if messages %}
    <div class="alert alert-danger my-3" role="alert">
    {% for message in messages %}
        <strong>{{ message.tags }}</strong>
        <ul><li>{{ message.message }}</li></ul>
    {% endfor %}
    </div>
    {% endif %}
...
```










#### 질문 삭제 기능 추가
1. 질문 상세 화면에 질문 삭제 버튼 추가 (템플릿 수정)   

question_detail.html 추가하기
```
...
{% if request.user == question.author %}
<div class="my-3">
    <a href="{% url 'pybo:question_modify' question.id  %}"
       class="btn btn-sm btn-outline-secondary">수정</a>
<!-- ------------------------------- [edit] -------------------------------- -->
    <a href="{% url 'first:question_delete' question.id  %}"  class="delete btn btn-sm btn-outline-secondary">삭제</a>


</div>
{% endif %}
...
```

2. 질문 템플릿에 삭제 알림 창 기능 추가

/question_detail.html 맨 마지막에 추가
```
...
<!-- $(document).ready(function()= 화면이 로드된 다음 자동으로 호출되는 제이쿼리 함수 -->
{% block script %}
<script type='text/javascript'>
 $(document).ready(function(){
    $(".delete").on('click', function() {
        if(confirm("정말로 삭제하시겠습니까?")) { #true라면 this(=이벤트가 발생한 객체(=delete))를 url로 넘겨준다 
            location.href = $(this).data('uri');
        }
    });
});
</script>
{% endblock %}
```

3. 질문 삭제 URL 매핑 추가

first/urls.py에 추가
```
path('question/delete/<int:question_id>/', views.question_delete, name='question_delete'),
```

4. 질문 삭제 함수 추가

first/views.py 맨 아래 추가
```
@login_required(login_url='common:login')
def question_delete(request, question_id):
    """
    질문 삭제 수정
    """
    question = get_object_or_404(Question, pk=question_id)
    if request.user != question.author:
        messages.error(request, '삭제권한이 없습니다')  #오류 발생
        return redirect('firtst:detail', question_id=question.id)
    question.delete()
    return redirect('first:index')
```

5. jQuery 실행을 위해 first/base.html 파일 수정

```
...
<!-- ------------------------------- [edit] -------------------------------- -->
<!-- 자바스크립트 Start -->
{% block script %}
{% endblock %}
<!-- 자바스크립트 End -->
<!-- ----------------------------------------------------------------------- -->
</body>
</html>
```




#### 답변 글 수정 함수 추가하기  
question_detail 템플릿을 answer 수정용 템플릿으로 사용하는 것은 적합하지 않으므로 답변 수정에 사용할 템플릿이 추가로 필요

1. answer 수정 버튼 추가하기

question_detail.html 추가
```
{% for answer in question.answer_set.all %}
<div class="card my-3">
    <div class="card-body">
        ...
<!-- ------------------------------- [edit] -------------------------------- -->
        {% if request.user == answer.author %}
        <div class="my-3">
            <a href="{% url 'pybo:answer_modify' answer.id  %}" 
               class="btn btn-sm btn-outline-secondary">수정</a>
        </div>
        {% endif %}
```

2. answer 수정 URL 매핑 추가하기

first/urls.py 에서 매핑 추가
```
path('answer/modify/<int:answer_id>/', views.answer_modify, name='answer_modify'),
```

3. answer 수정 함수 추가하기

first/views.py 에서 추가
```
@login_required(login_url='common:login')
def answer_modify(request, answer_id):
    """
    pybo 답변수정
    """
    answer = get_object_or_404(Answer, pk=answer_id)
    if request.user != answer.author:
        messages.error(request, '수정권한이 없습니다')
        return redirect('pybo:detail', question_id=answer.question.id)

    if request.method == "POST":
        form = AnswerForm(request.POST, instance=answer)
        if form.is_valid():
            answer = form.save(commit=False)
            answer.author = request.user
            answer.modify_date = timezone.now()
            answer.save()
            return redirect('pybo:detail', question_id=answer.question.id)
    else:
        form = AnswerForm(instance=answer)
    context = {'answer': answer, 'form': form}
    return render(request, 'pybo/answer_form.html', context)
```

4. answer 수정 템플릿 생성

templates/firts/answer_form.html로 html 생성하기

```
{% extends 'base.html' %}

{% block content %}
<div class="container my-3">
    <form method="post" class="post-form">
        {% csrf_token %}
        {% include "form_errors.html" %}
        <div class="form-group">
            <label for="content">답변내용</label>
            <textarea class="form-control" name="content" id="content" 
                      rows="10">{{ form.content.value|default_if_none:'' }}</textarea>
        </div>
        <button type="submit" class="btn btn-primary">저장하기</button>
    </form>
</div>
{% endblock %}
```




#### 답변 글 삭제 함수 추가하기
1. answer 삭제 버튼 추가하기  

question_detail.html 추가  

2. answer 삭제 URL 매핑 추가하기  

first/urls.py 에서 매핑 추가

3. answer 수정 함수 추가하기  

first/views.py 에서 추가

4. answer 수정 템플릿 생성  

templates/firts/answer_form.html로 html 생성하기

#### 댓글 작성/수정/삭제 기능 구현
1. Comment 모델 (first/models.py)

models.py 맨 아래에 추가하기
```
class Comment(models.Model):
    author = models.ForeignKey(User, on_delete=models.CASCADE)
    content = models.TextField()
    create_date = models.DateTimeField()
    modify_date = models.DateTimeField(null=True, blank=True)
    question = models.ForeignKey(Question, null=True, blank=True, on_delete=models.CASCADE)
    answer = models.ForeignKey(Answer, null=True, blank=True, on_delete=models.CASCADE)
```

2. 모델을 수정했으므로 makemigrations, migrate 명령 실행하기

(mysite) c:\projects\mysite>python manage.py makemigrations  
(mysite) c:\projects\mysite>python manage.py migrate

3. (question_detail) 템플릿에 댓글 목록과 댓글 입력 링크 추가  

question_detail.html 추가
```
...
    <a href="{% url 'first:question_delete' question.id  %}"  class="delete btn btn-sm btn-outline-secondary">삭제</a>
             </div>
        {% endif %}
<!--  질문 댓글 영역 시작 -->
            {% if question.comment_set.count > 0 %}  <!--  질문 댓글이 있으면 시작 -->
             <div class="mt-3">
            {% for comment in question.comment_set.all %}  <!--  질문 댓글 전체를 가져옴 -->
            <div class="comment py-2 text-muted">
                <span style="white-space: pre-line;">{{ comment.content }}</span>
                <span>
            - {{ comment.author }}, {{ comment.create_date }}  <!--  누가 언제 작성했는지 -->
                {% if comment.modify_date %}
            (수정:{{ comment.modify_date }})
            {% endif %}
                 </span>
                 {% if request.user == comment.author %}   <!--  작성자와 사용자가 같다면 수정, 삭제 링크가 보이도록 설정 -->
                 <a href="{% url 'first:comment_modify_question' comment.id  %}" class="small">수정</a>,
                 <a href="#" class="small delete" data-uri="{% url 'first:comment_delete_question' comment.id  %}">삭제</a>
                 {% endif %}
        </div>
        {% endfor %}
    </div>
    {% endif %}
    <div>
        <a href="{% url 'first:comment_create_question' question.id  %}"  <!--  댓글을 작성할 수 있는 링크 추가 -->
           class="small"><small>댓글 추가 ..</small></a>
    </div>
            <!--  질문 댓글 영역 끝  -->
```

4. 질문 댓글 URL 매핑 추가하기

first/urls.py 에 추가

```
path('comment/create/question/<int:question_id>/', views.comment_create_question, name='comment_create_question'),
    path('comment/modify/question/<int:comment_id>/', views.comment_modify_question, name='comment_modify_question'),
    path('comment/delete/question/<int:comment_id>/', views.comment_delete_question, name='comment_delete_question'),
```

5. 질문 댓글 CommentForm을 작성

first/forms.py 추가  
```
from pybo.models import Question, Answer, Comment #Comment만 추가
...

class CommentForm(forms.ModelForm):
    class Meta:
        model = Comment
        fields = ['content']
        labels = {
            'content': '댓글내용',
        }
```

6. 질문 댓글 등록 함수 추가하기

first/views.py 파일에 댓글 등록 함수 comment_create_question 맨 아래에 추가
```
from .forms import QuestionForm, AnswerForm, CommentForm  #CommentForm 만 추가
...
#맨 아래
@login_required(login_url='common:login')
def comment_create_question(request, question_id):
    """
    질문 댓글 등록
    """
    question = get_object_or_404(Question, pk=question_id)
    if request.method == "POST":
        form = CommentForm(request.POST)
        if form.is_valid():
            comment = form.save(commit=False)
            comment.author = request.user
            comment.create_date = timezone.now()
            comment.question = question
            comment.save()
            return redirect('first:detail', question_id=question.id)
    else:
        form = CommentForm()
    context = {'form': form}
    return render(request, 'first/comment_form.html', context)
```

7. 질문 댓글 수정 함수 추가하기

```
@login_required(login_url='common:login')
def comment_modify_question(request, comment_id):
    """
    질문 댓글 수정
    """
    comment = get_object_or_404(Comment, pk=comment_id)
    if request.user != comment.author:
        messages.error(request, '댓글수정권한이 없습니다')
        return redirect('first:detail', question_id=comment.question.id) #comment로 넘어온 id를 넘겨야 한다

    if request.method == "POST":
        form = CommentForm(request.POST, instance=comment)
        if form.is_valid():
            comment = form.save(commit=False)
            comment.author = request.user
            comment.modify_date = timezone.now()
            comment.save()
            return redirect('first:detail', question_id=comment.question.id) #comment가 참조하는 id가 넘어와야 한다
    else:
        form = CommentForm(instance=comment)
    context = {'form': form}
    return render(request, 'first/comment_form.html', context)
```

8. 질문 댓글 삭제 함수 추가하기

```
@login_required(login_url='common:login')
def comment_delete_question(request, comment_id):
    """
    질문댓글삭제
    """
    comment = get_object_or_404(Comment, pk=comment_id)
    if request.user != comment.author:
        messages.error(request, '댓글삭제권한이 없습니다')
        return redirect('first:detail', question_id=comment.question.id)
    else:
        comment.delete()
    return redirect('first:detail', question_id=comment.question.id)
```

9. 댓글 입력 템플릿 작성하기

templates/first/comment_form.html 생성하기  
생성 후 추가
```
{% extends 'base.html' %}

{% block content %}
<div class="container my-3">
    <h5 class="border-bottom pb-2">댓글등록하기</h5>
    <form method="post" class="post-form my-3">
        {% csrf_token %}
        {% include "form_errors.html" %}
        <div class="form-group">
            <label for="content">댓글내용</label>
            <textarea class="form-control"name="content" id="content" 
                      rows="3">{{ form.content.value|default_if_none:'' }}</textarea>
        </div>
        <button type="submit" class="btn btn-primary">저장하기</button>
    </form>
</div>
{% endblock %}
```
